# 유튜브 채널 데이터 크롤링

본 Jupyter Notebook은 유튜브 데이터를 수집하기 위해 사용됩니다.  
하지만 현재 유튜브에서는 크롤링을 막아 놓아, 데이터를 수집하는 것이 불가능합니다.  
유튜브의 데이터 접근 방식이 변경됨에 따라 기존의 크롤링 기법으로는 원하는 정보를 얻기 어려운 상황입니다.

## 주의사항

### 1. 크롤링 제한
- 현재 유튜브에서는 크롤링을 막아 놓아, 데이터를 수집하는 것이 불가능합니다.  
유튜브의 데이터 접근 방식이 변경됨에 따라 기존의 크롤링 기법으로는 원하는 정보를 얻기 어려운 상황입니다.

### 2. 회사 보안 정책
- 이 코드 예시는 교육 목적으로 제공된 것이며, 실제 운영 코드와 다를 수 있습니다.  
회사의 보안 정책에 따라 실제 운영 환경에서는 특정 보안 조치가 필요할 수 있습니다.  
따라서 이 코드를 직접 운영 환경에서 사용하기 전, 반드시 내부 규정 및 보안 절차를 확인해야 합니다.

---

### 추가 정보
- **유튜브 데이터 API**: 필요한 정보를 수집하기 위해 [YouTube Data API](https://developers.google.com/youtube/v3) 사용을 고려해 보세요.
- **법적 고지**: 웹 크롤링은 웹사이트의 이용 약관을 준수해야 합니다.


In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

In [ ]:
# 크롬 드라이버 설정 (크롬 드라이버 경로 수정 필요)
chrome_driver_path = 'path/to/chromedriver'  # 크롬 드라이버 경로를 설정하세요.
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

In [ ]:
def get_channel_data(channel_url, user_id):
    """
    주어진 유튜브 채널 URL에서 채널 정보를 수집하는 함수입니다.

    Args:
        channel_url (str): 유튜브 채널의 URL
        user_id (str): 사용자의 ID (외부에서 주어짐)

    Returns:
        dict: 채널 정보가 포함된 딕셔너리
    """
    # 유튜브 채널 페이지 열기
    driver.get(channel_url)
    time.sleep(2)  # 페이지 로드 대기

    # 페이지의 HTML 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 채널 정보 수집을 위한 딕셔너리 초기화
    channel_data = {}

    # 채널 ID 추출
    channel_id_part = channel_url.split('/')[-1]
    channel_data['CHANNEL_ID'] = channel_id_part

    # 채널명 추출
    channel_name_element = soup.find('meta', {'property': 'og:title'})
    channel_data['CHANNEL_NAME'] = channel_name_element['content'] if channel_name_element else None

    # 채널 설명 추출
    channel_description_element = soup.find('meta', {'name': 'description'})
    channel_data['CHANNEL_DESCRIPTION'] = channel_description_element['content'] if channel_description_element else None

    # 채널 태그 추출 (유튜브 채널 태그는 메타 데이터에 존재할 수 있음)
    tag_elements = soup.find_all('meta', {'name': 'keywords'})
    channel_data['CHANNEL_TAGS'] = tag_elements[0]['content'].split(',') if tag_elements else []

    # 주로 사용된 키워드
    channel_data['MAINLY_USED_KEYWORDS'] = ""

    # 주로 사용된 태그
    channel_data['MAINLY_USED_TAGS'] = ""

    # 채널 국가 추출 (일반적으로 채널 국가가 명시된 메타 데이터는 존재하지 않음)
    country_element = soup.find('meta', {'itemprop': 'nationality'})
    channel_data['CHANNEL_COUNTRY'] = country_element['content'] if country_element else None

    # 채널 링크
    channel_data['CHANNEL_LINK'] = channel_url

    # 채널 생성일 추출 (메타 정보로부터 직접 추출되지 않으므로, 비디오 업로드 기록 등으로 유추해야 할 수 있음)
    channel_since_element = soup.find('yt-formatted-string', {'id': 'subscriber-count'})
    channel_data['CHANNEL_SINCE'] = channel_since_element.text if channel_since_element else None

    # 채널 클러스터 (해당 정보는 별도로 설정해줘야 함)
    channel_data['CHANNEL_CLUSTER'] = None

    # 크롤링 일시
    channel_data['CRAWLED_DATE'] = time.strftime("%Y-%m-%d %H:%M:%S")

    # 사용자 ID
    channel_data['USER_ID'] = user_id

    # 채널 ID 부분
    channel_data['channel_id_part'] = channel_id_part

    # 채널 썸네일 URL 추출
    thumbnail_element = soup.find('img', {'id': 'img', 'alt': channel_data['CHANNEL_NAME']})
    channel_data['CHANNEL_THUMBNAIL'] = thumbnail_element['src'] if thumbnail_element else None

    return channel_data


In [ ]:
# 채널 URL 리스트
channel_urls = [
    'https://www.youtube.com/channel/CHANNEL_ID_1',  # 실제 채널 URL을 입력하세요.
    'https://www.youtube.com/channel/CHANNEL_ID_2'   # 실제 채널 URL을 입력하세요.
]


In [ ]:

# 채널 데이터 수집을 위한 리스트 초기화
channel_data_list = []

# 각 채널 URL에 대해 데이터 수집
for url in channel_urls:
    channel_data = get_channel_data(url)  # 채널 데이터 수집
    channel_data_list.append(channel_data)  # 수집한 데이터 리스트에 추가

# 수집한 채널 데이터를 확인
for channel in channel_data_list:
    print(channel)

In [ ]:
# 수집된 데이터를 DataFrame으로 변환
df = pd.DataFrame(channel_data_list)

# DataFrame을 CSV 파일로 저장
df.to_csv('../data/channel_sample.csv', index=False, encoding='utf-8-sig')


In [ ]:
# 드라이버 종료
driver.quit()